In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

from selenium import webdriver
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException
#from selenium.webdriver.common.by import By



url = 'https://www.immoweb.be/en/search/castle/for-sale?countries=BE&page=1&orderBy=relevance'

driver = webdriver.Firefox()
driver.implicitly_wait(1)
driver.get(url)

element = driver.find_element(by='id', value='uc-btn-accept-banner')
element.click()

soup_file = driver.page_source
soup = BeautifulSoup(soup_file)


driver.close()


In [5]:
def cooking_soup(url):
   driver = webdriver.Firefox()
   driver.implicitly_wait(3)
   driver.get(url)

   #element = driver.find_element(by='id', value='uc-btn-accept-banner')
   #element.click()

   soup_file = driver.page_source
   soup = BeautifulSoup(soup_file)
   driver.close()
   return soup


def url_getter(soup):
   list_of_urls = []
   first_tags = soup.find_all('li', attrs={'class': 'search-results__item'})
   houses = []
   for tag in first_tags:
      houses += tag.find_all('a', attrs={'class': 'card__title-link'})
   for house in houses:
      list_of_urls.append(house.get('href'))
   return list_of_urls

In [18]:
def get_next_page(soup):
        button = soup.find('ul', attrs={'class': 'pagination'})
        ref = button.find('a', 'pagination__link pagination__link--next button button--text button--size-small')
        if ref:
            return ref.get('href')
        else:
            return None
    
    

In [37]:
url = 'https://www.immoweb.be/en/search/duplex/for-sale?countries=BE&page=1&orderBy=relevance'
all_url = []
new_url = url
while new_url and (len(all_url)<100):
    soup = cooking_soup(new_url)
    new_url = get_next_page(soup)
    list = url_getter(soup)
    all_url += list
url_list = pd.Series(all_url)
url_list.to_csv('urls')


In [39]:
sub_types = ['bungalow', 'castle', 'country-cottage', 'apartment-block', 'town-house', 'chalet', 'farmhouse', 'exceptional-property','mixed-use-building', 'mansion', 'villa', 'manor-house','pavilion', 'other-property', 'duplex', 'triplex', 'studio', 'penthouse', 'loft']

In [40]:
def get_all_urls(name: str):
    all_url = []
    new_url = f"https://www.immoweb.be/en/search/{name}/for-sale?countries=BE&page=1&orderBy=relevance"
    while new_url and (len(all_url)<10):
        soup = cooking_soup(new_url)
        new_url = get_next_page(soup)
        list = url_getter(soup)
        all_url += list
    url_list = pd.Series(all_url)
    url_list.to_csv(f"urls_{name}")

for i in sub_types[:3]:
    get_all_urls(i)

In [41]:
from threading import Thread

class urlScrapper(Thread):

    def __init__(self, name):
        super().__init__(self)
        self.name = name
    
    def run(self):
        get_all_urls(self.name)

In [43]:
dict = {}
for name in sub_types[:3]:
    dict[name] = urlScrapper(name)

for name in sub_types[:3]:
    dict[name].start()

AssertionError: group argument must be None for now